In [ ]:
import numpy as np
import plotly.express as px
import matplotlib.pyplot as plt
import scipy.stats as stats
from tqdm import tqdm

import finito.simulator as sim

In [ ]:
SEED = 42
START = np.datetime64("2022-01-01T00:00:00")
DURATION = np.timedelta64(15, "s")
DELTA = np.timedelta64(10, "ms")

# Type of processes

## Wiener process

Z is a Wiener process if
1. $\Delta Z = \varepsilon \cdot\sqrt{\Delta{t}}$, where $\varepsilon \sim \Phi{(0,1)}$
2. $\Delta Z$ are independent for small ${\Delta{t}}$ intervals


## Generalized Wiener process
X is a generalized Wiener process if
$dX = a \,dt + b \,dZ$, where $Z$ is a Wiener process

In [ ]:
def generateWiener(
    dt: np.timedelta64 = DELTA,
    z0=0,
    T: np.timedelta64 = DURATION,
    seed=SEED,
    disable_tqdm=False,
):
    """Generate Wiener process

    Each time creates new process. Doesn't change resolution of time.
    """

    np.random.seed(seed)
    Z = np.full(T // dt, z0, dtype=float)
    delta_t = dt / np.timedelta64(1, "s")
    for i in tqdm(
        range(1, Z.shape[0]), "Generating Wiener process", disable=disable_tqdm
    ):
        randomVal = stats.norm.rvs(0, np.sqrt(delta_t))
        Z[i] = Z[i - 1] + randomVal * np.sqrt(delta_t)
    return Z

In [ ]:
px.line(sim.generateGeneralWiener(3, 0.3, dt=DELTA, T=DURATION)).show()

In [ ]:
for num_ticks in [100 * DELTA, 10 * DELTA, DELTA]:
    Z = generateWiener(num_ticks)
    px.line(Z).show()

# Gaussian mixture

In [ ]:
from finito.gauss import GaussianMixture
from scipy.stats import norm

In [ ]:
a = [0, 0, 0]
b = [1, 5, 2]


gm1 = GaussianMixture(a=a, b=b)
x_grid = np.linspace(-7, 7, 100)
cdf_values = gm1.cdf(x_grid)
print(
    "Plots are different time to time due to random weights generation.",
    f"\nCurrent weights: {gm1._weights.flatten()}",
)

pdf_values = gm1.pdf(x_grid)

plt.figure(figsize=(8, 5))
plt.plot(x_grid, pdf_values, label="GMM PDF", color="black")

# Approximate samples of gaussian mixture with just one normal law
samples = gm1.sample(500)
params = norm.fit(samples)
plt.plot(x_grid, norm(*params).pdf(x_grid), label="norm PDF", color="blue")
plt.title("PDF of Gaussian Mixture Model")
plt.xlabel("x")
plt.ylabel("PDF")
plt.legend()
plt.grid()
plt.show()

As we can see even that gaussians have the same mean, that leads 
to pdf and cdf plots form similar to basic normal distribution, the differences 
in standard deviations pays crucial role.

Check cdf's of real data (BTC for instance) and you'll see similar pictures
